In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import os
import torch
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from keras.optimizers import SGD, Adam, Adadelta, RMSprop, Adagrad, Nadam, Ftrl
from keras.regularizers import l2
import numpy as np
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler


In [2]:
# Load the training data
train_data = pd.read_csv('../../2nd-Comp-Data/train.csv')
test_data = pd.read_csv('../../2nd-Comp-Data/test.csv')

used = []

# Extract features and target variable
X = train_data.drop('price_doc', axis=1)
y = train_data['price_doc']
X_test = test_data.drop(['row ID'], axis=1)

In [3]:
X = X.drop('sub_area', axis=1)
X_test = X_test.drop('sub_area', axis=1)
used.append('Removed sub_area')

In [4]:
X = pd.get_dummies(X)
X_test = pd.get_dummies(X_test) 
used.append('OneHot Encoding')

In [5]:
# from sklearn.calibration import LabelEncoder

# categorical_columns = X.select_dtypes(include=['object']).columns.tolist()
# print("Train: Categorical columns:", categorical_columns)

# label_encoder = LabelEncoder()

# for column in categorical_columns:
#     X[column] = label_encoder.fit_transform(X[column])

# categorical_columns_test = X_test.select_dtypes(include=['object']).columns.tolist()
# print("Test: Categorical columns:", categorical_columns_test)

# label_encoder = LabelEncoder()

# for column in categorical_columns_test:
#     X_test[column] = label_encoder.fit_transform(X_test[column])

# used.append('Label Encoding')

In [6]:
# # drop all columns in X_train with dtypes object
# for col in X.columns:
#     if X[col].dtype == 'object':
#         X.drop(col, axis=1, inplace=True)

# # drop all columns in X_test with dtypes object
# for col in X_test.columns:
#     if X_test[col].dtype == 'object':
#         X_test.drop(col, axis=1, inplace=True)

# used.append("Removed Object Dtypes")

In [7]:
# X = X[['full_sq', 'floor', 'build_count_monolith', 'industrial_km', 'trc_sqm_500',
#  'mosque_count_500', 'leisure_count_500', 'office_sqm_1000',
#  'cafe_count_1000_price_high', 'leisure_count_1000', 'power_transmission_line_km', 'big_market_km', 'public_healthcare_km', 'workplaces_km']]
# X_test = X_test[['full_sq', 'floor', 'build_count_monolith', 'industrial_km', 'trc_sqm_500',
#  'mosque_count_500', 'leisure_count_500', 'office_sqm_1000',
#  'cafe_count_1000_price_high', 'leisure_count_1000', 'power_transmission_line_km', 'big_market_km', 'public_healthcare_km', 'workplaces_km']]

# used.append("Forward Feature Selection (n=14)")

In [8]:
# from sklearn.feature_selection import VarianceThreshold


# threshold_value = 4000000
# variance_filter = VarianceThreshold(threshold=threshold_value)

# X = variance_filter.fit_transform(X)
# X_test = variance_filter.transform(X_test)

# used.append("Variance Based Feature Selection")

In [9]:
# import statsmodels.api as sm

# # Add a constant term to the feature matrix
# X_with_const = sm.add_constant(X)

# # Fit a linear regression model
# model = sm.OLS(y, X_with_const).fit()

# # Get p-values for each feature
# p_values = model.pvalues[1:]  # Exclude the constant term

# # Set your desired threshold for p-value
# threshold = 0.00001

# # Filter features based on p-value
# selected_features = p_values[p_values < threshold].index

# # Display selected features
# print("Selected Features:")
# print(selected_features)
# print(len(selected_features))

# # Select columns in the DataFrame
# X = X[selected_features]
# X_test = X_test[selected_features]

# used.append("P-Value Based Feature Selection")

In [10]:
from sklearn.ensemble import GradientBoostingRegressor
rf_model = GradientBoostingRegressor(n_estimators=20, learning_rate=0.1, max_depth=10, random_state=42, loss='squared_error', verbose=2)
used.append('GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=42, loss=\'squared_error\')')

# Train the model to get feature importances
rf_model.fit(X, y)

# Feature importance analysis
feature_importances = rf_model.feature_importances_

# Create a DataFrame with feature names and their importance scores
feature_importance_df = pd.DataFrame({'Feature': X.columns, 'Importance': feature_importances})

# Sort the features based on importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Select the top N most important features
top_n_features = 10  # You can choose the number of top features based on your preference
selected_features = feature_importance_df.head(top_n_features)['Feature'].tolist()

# Subset the original data to include only the selected features
X = X[selected_features]
X_test = X_test[selected_features]

used.append("GradientBoostingRegressor Feature Importance Based Feature Selection")

      Iter       Train Loss   Remaining Time 
         1 415119387722197.3125           17.24m
         2 364809917278342.2500           14.86m
         3 324146418486510.6875           12.92m
         4 290631658161203.1250           11.71m
         5 263151698839354.8125           10.56m
         6 241173886748647.9375            9.55m
         7 222901692785901.3438            8.67m
         8 207291936492291.4375            7.82m
         9 194542517844857.0312            7.12m
        10 184328232441338.9688            6.49m
        11 175952942128201.4375            5.79m
        12 167688159447018.0938            5.10m
        13 161474942381453.2188            4.42m
        14 155000892080855.8438            3.75m
        15 150427678394866.6875            3.11m
        16 145758372837330.1562            2.47m
        17 142858808428190.6562            1.84m
        18 139621163137247.9219            1.22m
        19 137131414069764.5000           36.36s
        20 134410183235

In [11]:
# pca = PCA(n_components=10)
# principalComponents = pca.fit_transform(X)
# X = pd.DataFrame(data = principalComponents)

# pca2 = PCA(n_components=10)
# principalComponents = pca2.fit_transform(X_test)
# X_test = pd.DataFrame(data = principalComponents)

# used.append('PCA (n=10)')

In [12]:
X = X.astype('float32')
X_test = X_test.astype('float32')
used.append("Converted All Columns To float32")

In [13]:
X = np.log1p(X)
X_test = np.log1p(X_test)
used.append('log Normalization')

In [14]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

In [15]:
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)
used.append("StandardScaler")

In [16]:
X_train.shape

(127054, 10)

In [17]:
# from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(2)#, interaction_only=True)
# X_train = poly.fit_transform(X_train)
# X_test = poly.fit_transform(X_test)
# used.append('PolynomialFeatures W/O Interaction')
# # used.append('PolynomialFeatures With Interaction')

In [18]:
# # save X_train to csv
# X_train.to_csv('train with poly w/o int.2.csv', index=False)
# X_test.to_csv('test with poly w/o int.csv', index=False)

In [19]:
# pca = PCA(n_components=200)
# principalComponents = pca.fit_transform(X_train)
# X_train = pd.DataFrame(data = principalComponents)

# pca2 = PCA(n_components=200)
# principalComponents = pca2.fit_transform(X_test)
# X_test = pd.DataFrame(data = principalComponents)

# used.append('PCA (n=200)')

In [20]:
features_used = X_train.shape[1]

In [21]:
from sklearn.ensemble import GradientBoostingRegressor
rf_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=42, loss='squared_error', verbose=2)
used.append('GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=42, loss=\'squared_error\')')

# Train the model
rf_model.fit(X_train, y_train)

# Make predictions on the validation set
val_predictions = rf_model.predict(X_val)

# Evaluate the model on the validation set
val_mse = mean_squared_error(y_val, val_predictions)
print(f'Validation Mean Squared Error: {val_mse}')

# Make predictions on the test set
predictions = rf_model.predict(X_test)


      Iter       Train Loss   Remaining Time 
         1 415297968164954.9375            1.53m
         2 365758662522569.3125            1.49m
         3 325590516028060.7500            1.46m
         4 292821300152088.0000            1.43m
         5 266467976235713.7188            1.41m
         6 244632586141804.3438            1.39m
         7 227162493540404.3438            1.63m
         8 212355496921519.6250            1.71m
         9 200748321905361.1250            1.69m
        10 191237535184521.9062            1.65m
        11 183218997940455.3125            1.61m
        12 176636649474690.6875            1.60m
        13 171521420643193.0000            1.60m
        14 167034734021593.7812            1.61m
        15 162892112011471.7500            1.59m
        16 159850834894818.8125            1.56m
        17 156820598170511.8438            1.54m
        18 154541012707439.4688            1.52m
        19 152830610361361.6562            1.50m
        20 150816867634

In [22]:
# Create a DataFrame with the results
submission_df = pd.DataFrame({'row ID': test_data['row ID'], 'price_doc': predictions.flatten()})

# Save the results to a CSV file
submission_df.to_csv('Day16.4.csv', index=False)

In [23]:
print("Features Used = " + str(features_used) + "\n")
for i in used:
    print(i)

Features Used = 10

Removed sub_area
OneHot Encoding
GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=42, loss='squared_error')
GradientBoostingRegressor Feature Importance Based Feature Selection
Converted All Columns To float32
log Normalization
StandardScaler
GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=10, random_state=42, loss='squared_error')
